In [1]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline, FeatureUnion 
from sklearn.base import BaseEstimator, TransformerMixin 
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold, RandomizedSearchCV 
from sklearn.linear_model import Perceptron, LogisticRegressionCV, RidgeClassifierCV, SGDClassifier, PassiveAggressiveClassifier, Lasso
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score,mean_absolute_error, confusion_matrix, silhouette_score
from sklearn.metrics import roc_auc_score,roc_curve, auc, classification_report,precision_score,recall_score,log_loss,f1_score
from sklearn.feature_selection import RFE
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.naive_bayes import GaussianNB, BernoulliNB, ComplementNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from scipy.stats import uniform, randint
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, BaggingClassifier, VotingClassifier, AdaBoostClassifier
from bayes_opt import BayesianOptimization
from lightgbm import LGBMClassifier
from imblearn.over_sampling import SMOTE
from sklearn import tree
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler, LabelEncoder, OneHotEncoder,OrdinalEncoder
from sklearn.preprocessing import MaxAbsScaler, RobustScaler, QuantileTransformer, PowerTransformer,minmax_scale,PolynomialFeatures
from sklearn.svm import SVC, LinearSVC
from sklearn.kernel_approximation import RBFSampler
from sklearn import tree
import pandas_bokeh
from sklearn.decomposition import PCA,KernelPCA
from sklearn.calibration import CalibratedClassifierCV
from numpy import mean, std
import pandas.testing as tm
from scipy import stats
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.inspection import permutation_importance
from sklearn.tree import ExtraTreeClassifier
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer, SimpleImputer, KNNImputer

from yellowbrick.features import PCA as PCA_YB
from yellowbrick.features.radviz import RadViz
from yellowbrick.features import pca_decomposition
from yellowbrick.features import Manifold
from yellowbrick.features import JointPlotVisualizer
from yellowbrick.classifier import ClassificationReport
from yellowbrick.classifier import PrecisionRecallCurve
from yellowbrick.classifier import ClassPredictionError
from yellowbrick.model_selection import LearningCurve
from yellowbrick.model_selection import CVScores
from yellowbrick.model_selection import FeatureImportances
from yellowbrick.features import ParallelCoordinates
from yellowbrick.model_selection import RFECV
from yellowbrick.classifier import ROCAUC


#other
from math import sqrt
import inspect
from matplotlib.font_manager import FontProperties
from scipy.stats import loguniform, uniform
from bokeh import io
import datetime
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
import eli5
%matplotlib inline

In [2]:
DATASET_URL = "https://gist.githubusercontent.com/YHYeoh/ad1a7f7170c72d621d05a70637540152/raw/5a6059c199e2c46d2f3d258f03d93cfea98e2749/marketing_campaign.csv"
data = pd.read_csv(DATASET_URL, sep = ';')

pd.set_option('plotting.backend','pandas_bokeh')

# data.fillna(method = "ffill", inplace = True)
data.isnull().values.any()


education_order = [['Basic', 'Graduation', 'Master', '2n Cycle', 'PhD']]
ordinal_encoder = OrdinalEncoder(categories=education_order)
enc = OneHotEncoder()


imr = IterativeImputer(random_state=42, max_iter=100, min_value=data['Income'].min())
imr = imr.fit(data[['Income']])
data['Income'] = imr.transform(data[['Income']]).ravel()


data["Education"] = (ordinal_encoder.fit_transform(data["Education"].values.reshape(-1, 1))).astype(int)
print(ordinal_encoder.categories_)

data_copy = data.copy()
marital_status_ohe = pd.get_dummies(data["Marital_Status"],prefix="Marital")
ohe_cols = marital_status_ohe.columns
data = pd.concat([data, marital_status_ohe], axis=1)

data.drop(["ID", 'Dt_Customer',"Z_CostContact","Z_Revenue","Marital_Status"], axis=1, inplace=True)

categorical = ['Marital_Status']
numerical = ['Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
       'Teenhome', 'Recency', 'MntWines', 'MntFruits', 'MntMeatProducts',
       'MntFishProducts', 'MntSweetProducts', 'MntGoldProds',
       'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases',
       'NumStorePurchases', 'NumWebVisitsMonth', 'AcceptedCmp3',
       'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1', 'AcceptedCmp2',
       'Complain']
numerical_no_bool = ['Education','Income', 'Kidhome', 'Teenhome', 'Recency', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth']

y = data.Response
print(y.value_counts())
X = data.drop(['Response'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size = 0.3)


[array(['Basic', 'Graduation', 'Master', '2n Cycle', 'PhD'], dtype=object)]
0    1906
1     334
Name: Response, dtype: int64


In [3]:

def getIterativeImputedIncome(data):
	imr = IterativeImputer(random_state=42, max_iter=100, min_value= data['Income'].min())
	imr = imr.fit(data[['Income']])
	data['Income'] = imr.transform(data[['Income']]).ravel()
	return data

def getKNNImputedIncome(data):
	imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
	data = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)
	return data

def getAgeFromDateBirth(data):
	year = datetime.datetime.now().year
	data['Year_Birth'] = data['Year_Birth'].apply(lambda x : year - x )
	return data

def extractFromDate(data):
	data['enroll_year'] = pd.DatetimeIndex(data.Dt_Customer).year
	data['enroll_month'] = pd.DatetimeIndex(data.Dt_Customer).month
	data['enroll_day'] = pd.DatetimeIndex(data.Dt_Customer).day
	data.drop(['Dt_Customer'], axis = 1, inplace= True)
	return data

def convertToDays(data):
	vfunc = np.vectorize(lambda x: (datetime.datetime.now() - x).days)
	data['Dt_Customer'] = vfunc(pd.DatetimeIndex(data.Dt_Customer).to_pydatetime())
	return data

def getBinnedIncome(data):
	data['Income'] = (pd.cut(data['Income'], bins=[0, 15000, 60000, 110000, 300000], labels=False, precision=0)).convert_dtypes()
	return data

def getNormalizedIncome(data):
	data = data[(np.abs(stats.zscore(data[['Income']])) < 3)]
	return data

def getNormalizedAndBinnedIncome(data):
	data = data[(np.abs(stats.zscore(data[['Income']])) < 3)]
	data['Income'] = pd.cut(data['Income'], bins=[0, 15000, 60000, 110000, 300000], labels=False, precision=0).convert_dtypes()
	return data

def getHypeYearBirth(data):
	hyper_year_birth = [
		data,
		getAgeFromDateBirth(data.copy()),
	]
	return hyper_year_birth



def getPreprocessingIncome(data):
	preprocessing_income = [
		getIterativeImputedIncome(data.copy()),
		getKNNImputedIncome(data.copy()),
		data.copy().fillna(method = "ffill"), #ffill
		data.copy().fillna(method = "bfill"), #bfill
		data.copy().fillna(data.mean()), #mean imputed
		data.copy().fillna(data.median()), #median imputed
		data.copy().dropna(subset=['Income'])
	]
	return preprocessing_income

def getHypeIncome(data):
	hyper_income = [
	data,
	getBinnedIncome(data.copy()),
	getNormalizedIncome(data.copy()),
	getNormalizedAndBinnedIncome(data.copy())
	]
	return hyper_income

In [7]:
#remove default unimportant column
unimportant = ["Year_Birth"]
# X_train_Kmeans = X_train_Kmeans.drop(columns=unimportant,axis=1)
# X_test_Kmeans = X_test_Kmeans.drop(columns=unimportant,axis=1)
# X_train_PCA_Kmeans = X_train_PCA_Kmeans.drop(columns=unimportant,axis=1)
# X_test_PCA_Kmeans = X_test_PCA_Kmeans.drop(columns=unimportant,axis=1)
X_train = X_train.drop(columns=unimportant,axis=1)
X_test = X_test.drop(columns=unimportant,axis=1)
y_train = y_train.drop(columns=['index'],axis=1)
y_test = y_test.drop(columns=['index'],axis=1)

In [8]:
y_train

994     0
2162    0
906     1
572     0
1877    0
       ..
1638    0
1095    0
1130    0
1294    0
860     0
Name: Response, Length: 1568, dtype: int64

In [9]:
# #add kmean column into all known data except x_train
# X_train_Kmeans["KMean"]=kmeanTrainColumn
# X_test_Kmeans["KMean"]=kmeanTestColumn
# X_train_PCA_Kmeans["KMean"]=kmeanTrainColumn
# X_test_PCA_Kmeans["KMean"]=kmeanTestColumn

In [10]:
# X_train_PCA_Kmeans

In [11]:
# #addPCA column
# kpcaCol = ["PCA1","PCA2","PCA3","PCA4","PCA5"]

# #PCA and Kmeans
# X_train_PCA_Kmeans[kpcaCol]=kpcaTrain
# X_test_PCA_Kmeans[kpcaCol]=kpcaTest

# #PCA only
# X_train_PCA = kpcaTrain
# X_test_PCA = kpcaTest

In [12]:
# rbf_sampler = RBFSampler(gamma = 5.0,n_components=100)
# X_train_rbf = rbf_sampler.fit_transform(X_train_PCA)
# X_test_rbf = rbf_sampler.transform(X_test_PCA)

In [13]:
# X_trainS, y_trainS = sampler.fit_resample(X_train, y_train.copy())
# # stratified(SMOTE) x with k means
# X_train_Kmeans, y_trainR = sampler.fit_resample(X_train_Kmeans, y_train.copy())
# # stratified(SMOTE) x with pca
# X_train_PCA, y_trainR = sampler.fit_resample(X_train_PCA, y_train.copy())
# # stratified(SMOTE) x with pca and k means
# X_train_PCA_Kmeans, y_trainR = sampler.fit_resample(X_train_PCA_Kmeans, y_train.copy())

In [14]:
# sampler = SMOTE()
# stratified(SMOTE) x
# stratified(SMOTE) x with k means
# stratified(SMOTE) x with pca
# stratified(SMOTE) x with pca and k means

#X_train, y_train.copy()
#X_train_Kmeans, y_train.copy()
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
oversampler = RandomOverSampler(sampling_strategy=0.5)
undersampler = RandomUnderSampler(sampling_strategy='majority')
X_trainOV,y_trainOV = oversampler.fit_resample(X_train, y_train)
X_trainUN, trainUN = undersampler.fit_resample(X_train, y_train)

In [21]:
# models = [
# 	#Lasso(),
# # 	SGDClassifier(max_iter = 1000, tol=1e-3,penalty = "elasticnet"),
# # 	LinearSVC(), 
# # 	GaussianProcessClassifier(),
# # 	ExtraTreesClassifier(), 
# # 	BernoulliNB(),
# 	LogisticRegressionCV(max_iter= 1200), 
# # 	RidgeClassifierCV(),
# 	SVC(kernel = 'linear',max_iter= -1), 
# # 	Perceptron(),
# # 	PassiveAggressiveClassifier(), 
# # 	DecisionTreeClassifier(), #no coef 
# # 	KNeighborsClassifier(),#no feat_import, use permutation_importance 
# # 	GaussianNB(), #no feat_import, use permutation_importance 
# 	LGBMClassifier(),#no coef 
# # 	RandomForestClassifier(), #no coef 
# # 	GradientBoostingClassifier(),#no coef 
# # 	PassiveAggressiveClassifier(), 
# # 	ExtraTreesClassifier(), #no coef 
# 	XGBClassifier(),
# 	AdaBoostClassifier(), #no coef
# # 	MLPClassifier() #mlp not working
# 	]






In [22]:
model_result= []
classes = ["no", "yes"]

for data in dataList:
    print(data['DataName'])
    xTrainData = data['X_train']
    yTrainData = data['y_train']
    for model in models:
        print(model.__class__.__name__)
        model_result.append(cross_validate(model,StratifiedKFold(),data['DataName'],xTrainData, yTrainData))
        overallClassificationReport(model,classes)
        if hasmethod(model, 'predict_proba'):
            ROC_Curve_Plot(model,X_test,y_test,"Overall "+model.__class__.__name__)
        if has_feature_imp(model) :
            overall_feature_importance(model,xTrainData, yTrainData)

import warnings
warnings.filterwarnings("default")

Unmodified Dataset
RandomForestClassifier


(313,) (313,)


KeyboardInterrupt: 

In [ ]:
# DATASET_URL = "https://gist.githubusercontent.com/YHYeoh/ad1a7f7170c72d621d05a70637540152/raw/5a6059c199e2c46d2f3d258f03d93cfea98e2749/marketing_campaign.csv"
# data = pd.read_csv(DATASET_URL, sep = ';')

# pd.set_option('plotting.backend','pandas_bokeh')

# # data.fillna(method = "ffill", inplace = True)
# data.isnull().values.any()


# education_order = [['Basic', 'Graduation', 'Master', '2n Cycle', 'PhD']]
# ordinal_encoder = OrdinalEncoder(categories=education_order)
# enc = OneHotEncoder()


# imr = IterativeImputer(random_state=42, max_iter=100, min_value=data['Income'].min())
# imr = imr.fit(data[['Income']])
# data['Income'] = imr.transform(data[['Income']]).ravel()


# data["Education"] = (ordinal_encoder.fit_transform(data["Education"].values.reshape(-1, 1))).astype(int)
# print(ordinal_encoder.categories_)

# data_copy = data.copy()
# marital_status_ohe = pd.get_dummies(data["Marital_Status"],prefix="Marital")
# ohe_cols = marital_status_ohe.columns
# data = pd.concat([data, marital_status_ohe], axis=1)

# data.drop(["ID", 'Dt_Customer',"Z_CostContact","Z_Revenue","Marital_Status"], axis=1, inplace=True)

# categorical = ['Marital_Status']
# numerical = ['Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
#        'Teenhome', 'Recency', 'MntWines', 'MntFruits', 'MntMeatProducts',
#        'MntFishProducts', 'MntSweetProducts', 'MntGoldProds',
#        'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases',
#        'NumStorePurchases', 'NumWebVisitsMonth', 'AcceptedCmp3',
#        'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1', 'AcceptedCmp2',
#        'Complain']
# numerical_no_bool = ['Education','Income', 'Kidhome', 'Teenhome', 'Recency', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth']

